# Assignment 2: Multi-Scale Pyramids

## General Information
**Submmision:** in pairs.

## Introduction
In this exercise you will practice working with image pyramids, and get some first-hand experience with two of their applications:
* **Image focusing:** Using two or more images taken with different planes of focus in order to get a multi-focus image.
* **Image mosaicing:** Stitching images together in a visually-appealing way.

The exercise involves coding with Python. If you're unfamiliar with python, you may use the following [tutorial](http://cs231n.github.io/python-numpy-tutorial/).

**Note:** some cells are read-only, and can't be modified. There are also some read-only empty cells, that contain hidden tests.

## Student details

### Student A
**Name:**
Idan Tankel


**Email:**
shakel002@gmail.com
idan.tankel@weizmann.ac.il

**ID:**
209056712

**Note**
This notebook was edited in Azure machine learning studio

## Tips and Remarks
- General
    - Prefer the use of built-in functions when possible, as they are much
faster (and extensively tested)
    - To avoid confusion, it is better to use the terms "finest level" and
"coarsest level", instead of "top level" and "bottom level"
    - Add proper documentation to your code
- Python tips
    - To sub-sample a vector, you can use this syntax: `vec[::2]` (this
will take only the even indices)
    - The following numpy methods may be useful, especially in the last
task: `where`, `argmax`, `which`

In [ ]:
from collections import namedtuple

import numpy as np
from scipy.ndimage import convolve
import ipywidgets as widgets

from utils import imread, imwrite as _imwrite, imshow as _imshow, rgb2grey

# 0 Warm-up

When testing computer vision algorithms, it's necessary to visualize the results and to evaluate them with your own eyes. To make sure that the image is presented as you would expect, it's important to convert it to integer representation. That's because different packages (`opencv`, `PIL`, `tensorflow`) handle floating-point images differently.

### 0.1.1 Implement `safe_normalize()`

In this exercise, we provide you most of the needed I/O methods. You only need to implement the `safe_normalize()` function, which is given image and bounds (the expected `min` and `max` values), and normalizes it to integers (`np.uint8`) in the range `[0, 255]`.

You **may not** assume that the values in `image` are in the expected `bounds`, as it usually happens with generated/interpolated images. You **should** explicitly handle this case, and map out-of-bound values to the nearest value in `bounds`.

In [ ]:
def safe_normalize(image, bounds):
    """Receive an `image` represented by float numbers, with given range `bounds`, and convert it to `uint8`

    Args:
        image (np.ndarray): the image
        bounds (Tuple[float, float]): tuple of `(minval, maxval)`,  minimum and maximum values of image
    
    Returns:
        np.ndarray: the normalized image, with `dtype=np.uint8`

    """
    # YOUR CODE HERE
    image = np.clip(image,bounds[0],bounds[1]) #Clip image bounds - values higher than the bounds will become the bounds 
    # and practically vanish
    image -= bounds[0] #Bring minimum to zero - the image pixels now will be in (0,bounds[1]-bounds[0])
    divide_by = bounds[1]-bounds[0]
    image /= divide_by #Divide by maximum
    image *= 255 #change to 255 high limit
    image = np.round(image) # make the values rounded by 0 decimals
    image = image.astype('uint8') #Change type
    return image 

In [ ]:
### PROVIDED FUNCTION
def imwrite(path, image, bounds=(0, 1), **kwargs):
    """Normalize `image` and save it to `path`"""
    image = safe_normalize(image, bounds)
    return _imwrite(path, image, **kwargs)


### PROVIDED FUNCTION
def imshow(image, bounds=(0, 1), **kwargs):
    """Normalize `image` and show it."""
    image = safe_normalize(image, bounds)
    return _imshow(image, **kwargs)


### PROVIDED FUNCTION
def imshow_tabs(images, tab_names, titles=None, kwargs=None):
    """Normalize `images`, and show them in tabs with the given tab names"""
    assert len(images) == len(tab_names)
    if not isinstance(titles, (list, tuple)):
        titles = [titles] * len(images)
    assert len(images) == len(titles)
    if kwargs is None:
        kwargs = {}
    n = len(images)
    if isinstance(kwargs, dict):
        kwargs = [kwargs for _ in range(n)]
    outputs = [widgets.Output() for _ in range(n)]
    boxes = widgets.Tab(children=outputs)
    for i, (output, image, tab_name, title, kw) in enumerate(zip(outputs, images, tab_names, titles, kwargs)):
        boxes.set_title(i, tab_name)
        with output:
            imshow(image, title=title, **kw)
    
    display(boxes)    

### PROVIDED FUNCTION
def imshow_hbox(images, titles=None, kwargs=None):
    """Normalize `images`, and show them side-by-side"""
    if kwargs is None:
        kwargs = {}
    if not isinstance(titles, (list, tuple)):
        titles = [titles] * len(images)
    assert len(images) == len(titles)

    n = len(images)
    if isinstance(kwargs, dict):
        kwargs = [kwargs for _ in range(n)]
    outputs = [widgets.Output() for _ in range(n)]
    boxes = widgets.HBox(children=outputs)
    for i, (output, image, title, kw) in enumerate(zip(outputs, images, titles, kwargs)):
        with output:
            imshow(image, title=title, **kw)
    
    display(boxes)
    
### PROVIDED FUNCTION
def imshow_vbox(images, titles=None, kwargs=None):
    """Normalize `images`, and show them side-by-side"""
    if kwargs is None:
        kwargs = {}
    if not isinstance(titles, (list, tuple)):
        titles = [titles] * len(images)
    assert len(images) == len(titles)

    n = len(images)
    if isinstance(kwargs, dict):
        kwargs = [kwargs for _ in range(n)]
    outputs = [widgets.Output() for _ in range(n)]
    boxes = widgets.VBox(children=outputs)
    for i, (output, image, title, kw) in enumerate(zip(outputs, images, titles, kwargs)):
        with output:
            imshow(image, title=title, **kw)
    
    display(boxes)

# 1 Image Pyramids

In this part of the exercise, you will implement basic image pyramid functions, following the paper "The Laplacian Pyramid as a Compact Image Code" by Burt and Adelson, 1983. The paper is included in this exercise, and can be found in the `papers/` direcory.

All of the tests will be run with the following parameters:

In [ ]:
depth = 5
a = 0.375

## 1.1 Gaussian Pyramids [30 Points]

### 1.1.1 Implement `pyramid_kernel()` [5 points]

Implement the `pyramid_kernel()` function, which creates the *Generating Kernel* depending on the parameter `a` (see page 533 in the paper).

In [ ]:
def pyramid_kernel(a):
    """Return the 5-by-5 generating kernel, given parameter `a`
    
    Args:
        a (float): the kernel parameter
    
    Returns:
        np.ndarray: 5-by-5 generating kernel
    """

    # YOUR CODE HERE
    w = np.array([0.25 - a/2.0, 0.25, a, 0.25, 0.25 - a/2.0]) #Equal contribution dictates this as the form of w
    kernel = np.outer(w, w) # this is the outer product function
    # kernel = np.matmul(w.transpose(),w)


    return kernel

In [ ]:
KERNEL_0375 = np.array([[1,  4,  6,  4, 1],
                        [4, 16, 24, 16, 4],
                        [6, 24, 36, 24, 6],
                        [4, 16, 24, 16, 4],
                        [1,  4,  6,  4, 1]]) / 256.0

np.testing.assert_array_equal(pyramid_kernel(a=0.375), KERNEL_0375)

### 1.1.2 Implement `pyramid_reduce()` [10 points]
Implement the `pyramid_reduce()` (see *REDUCE*, page 533).

You're provided with `conv2d()`, and with the `pyramid_expand()` function (see *EXPAND*, page 534).

In [ ]:
### PROVIDED FUNCTION
def conv2d(image, kernel):
    """Convolve image with 2D kernel
    
    Args:
        image (np.ndarray): base image
        kernel (np.ndarray): kernel to convolve with `image`
    
    Returns:
        np.ndarray: `image` convolved with `kernel`
    """

    if image.ndim == 3 and kernel.ndim == 2:
        kernel = np.expand_dims(kernel, -1)
    
    return convolve(image, kernel, mode='mirror')

In [ ]:
def pyramid_reduce(image, kernel):
    """Reduce an image given a kernel
    
    See:
        "The Laplacian Pyramid as a Compact Image Code" by Burt & Adelson (1983)
    
    Args:
        image (np.ndarray): image to reduce, of size (2*h, 2*w)
        kernel (np.ndarray): kernel to use for the reduction (usually 5x5)
    
    Returns:
        np.ndarray: reduced image, of size (h, w)
    """
    # YOUR CODE HERE
    # first we will need to convolve the image with the kernel and than to take only 2 
    convedIm = conv2d(image,kernel) #Convolution with the kernel in order to blur the image
    return convedIm[::2,::2] #downsampling by a factor of 2

In [ ]:
### PROVIDED FUNCTION
def pyramid_expand(image, kernel, out_size=None):
    """Expand an image given a kernel
        
    Args:
        image (np.ndarray): image to expand, of size (h, w)
        kernel (np.ndarray): kernel to use for the expansion (usually 5x5)
        out_size (Tuple[int, int], optional): the expected shape of the returned image
        
    Returns:
        np.ndarray: expanded image, of size (2*h, 2*w)
    """
    h, w, *c = image.shape
    outh, outw = (2 * h, 2 * w) if out_size is None else out_size
    assert abs(outh - 2 * h) <= 1 and abs(outw - 2 * w) <= 1, "`out_size` should be close to `(2*h, 2*w)`"
    new_image = np.zeros((outh, outw, *c))    
    new_image[::2, ::2, ...] = image
    return conv2d(new_image, 4 * kernel)

In [ ]:
### THIS CELL CONTAINS HIDDEN TESTS

### 1.1.3 Implement `gaussian_reduce()` [10 points]

Implement `gaussian_reduce()`, using your previously implemented methods.

In [ ]:
def gaussian_reduce(image, kernel, depth):
    """Generate a Gaussian Pyramid from `image`, using `kernel` for reduction
    
    The pyramid includes the original image (level 0), followed by `depth` reductions
    
    This function uses `pyramid_reduce` as it's main act
    
    Args:
        image (np.ndarray): image to reduce
        kernel (np.ndarray): kernel to use for the reduction (usually 5x5)
        depth (int): how many reductions to apply
    
    Returns:
        List[np.ndarray]: List of `depth + 1` pyramid levels, where the first level is the original image
    """
    # YOUR CODE HERE
    to_return = [] #reduce is a list which contains the levels
    temp_layer = image # the fitst layer is the same as the original image 
    for i in range(depth+1):
        to_return.append(temp_layer) #Append the next level of the pyramid
        temp_layer = pyramid_reduce(temp_layer,kernel) # go ahead by one layer
    return to_return

In [ ]:
GaussianPyramidExample = namedtuple('GaussianPyramidExample', 'name path')
test_examples = [
    GaussianPyramidExample('Weizmann', 'data/pyramids/weizmann.png'),
    GaussianPyramidExample('Lenna', 'data/pyramids/lenna.png'),
    GaussianPyramidExample('Einstein', 'data/pyramids/einstein.png'),
]
examples_outputs = widgets.Accordion(children=[widgets.Output() for _ in range(len(test_examples))])

kernel = pyramid_kernel(a)
for i, example in enumerate(test_examples):
    # set example name
    examples_outputs.set_title(i, example.name)
    
    # plot inside example's area
    with examples_outputs.children[i]:
        im = imread(example.path)
        pyramid = gaussian_reduce(im, kernel, depth=depth)
        titles = ['Level %d' % i for i in range(len(pyramid))]
        imshow_vbox(pyramid, titles=titles)
        
display(examples_outputs)

### 1.1.4 Theoretical Question [5 points]

What is the purpose of the Gaussian filter in the pyramid construction? What would happen if only down-sampling was used?

If we were to simply down sample, we will lose a lot of data in the image. For instance, if the "image" is 25 1D (I know it's not so realistic but this is for the concept) then after a few iterations - first the number indexed [3,8,13,18,23] will stay, and than only the number of the 13th place will stay - a loss of most of the original information

## 1.2 Laplacian Pyramids [20 Points]



### 1.2.1 Implement `laplacian_reduce()` [10 points]

Implement `laplacian_reduce()`, using your previously implemented methods, and the provided `pyramid_expand()` method.

In [ ]:
def laplacian_reduce(image, kernel, depth):
    """Generate a Laplacian Pyramid from `image`, using `kernel` for reduction
    
    Args:
        image (np.ndarray): image to reduce
        kernel (np.ndarray): kernel to use for the reduction (usually 5x5)
        depth (int): how many reductions to apply
    
    Returns:
        List[np.ndarray]: List of `depth + 1` pyramid levels
    """
    
    gaussian_reduced = gaussian_reduce(image,kernel,depth) # first do a gaussian reduce
    to_return = []
    for i,image in enumerate(gaussian_reduced[:-1]):
        # the last element is append by itself - without any change
        # the other layers are computed by latters layers
        expend = pyramid_expand(gaussian_reduced[i+1],kernel,image.shape[:2]) #The expansion of the next level image
        # the out size is the image shape trimmed
        # Layer1 = g1 - expend(g2) where G1 - gauss1, G2 gauss 2
        to_return.append(image - expend) #Append
    to_return.append(gaussian_reduced[-1]) #Append last level by itself
    # the last element is in -1
    return to_return

In [ ]:
LaplacianPyramidExample = namedtuple('LaplacianPyramidExample', 'name path')
test_examples = [
    LaplacianPyramidExample('Weizmann', 'data/pyramids/weizmann.png'),
    LaplacianPyramidExample('Lenna', 'data/pyramids/lenna.png'),
    LaplacianPyramidExample('Einstein', 'data/pyramids/einstein.png'),
]
examples_outputs = widgets.Accordion(children=[widgets.Output() for _ in range(len(test_examples))])

kernel = pyramid_kernel(a)
for i, example in enumerate(test_examples):
    # set example name
    examples_outputs.set_title(i, example.name)
    
    # plot inside example's area
    with examples_outputs.children[i]:
        im = imread(example.path)
        pyramid = laplacian_reduce(im, kernel, depth=depth)
        titles = ['Level %d' % i for i in range(len(pyramid))]
        kwargs = ([{'bounds': (-1, 1)} for _ in range(len(pyramid) - 1)] +  # bounds of laplacian layer
                  [{'bounds': (0, 1)}])                                     # bounds of gaussian layer
        
        imshow_vbox(pyramid, titles=titles, kwargs=kwargs)
        
display(examples_outputs)

### 1.2.2 Theoretical Question [3 points]

What are the bounds for pixels at each level in the Laplaican pyramid? What is the meaning of bright/dark pixels that we see?

As it shown in "Laplasian Pyramids as a compact image code" the values are located around zero and they should be 

$values\in(-1,1)$

As further the pixels from grey (i.e extreme cases like black and white) the reconstruction of the pyramid is not working so well

### 1.2.3 Implement `laplacian_expand()` [7 points]

Implement `laplacian_expand()` that reconstructs an image from its Laplacian Pyramid.

In [ ]:
def laplacian_expand(pyramid, kernel):
    """Reconstruct an image from a Laplacian Pyramid
    
    Args:
        pyramid (List[np.ndarray]): list of `depth + 1` pyramid levels
        kernel (np.ndarray): kernel that was used to create `pyramid`
    
    Returns:
        np.ndarray: reconstructed image
    """
    # YOUR CODE HERE
    while len(pyramid) > 1:
        pyramidion = pyramid.pop() #the last layer in the pyramid - the pyramidion
        expended = pyramid_expand(pyramidion,kernel,pyramid[-1].shape[:2]) #Reconstruct the last layer - from the pyramidion the kernel
        # the requested shape is the shape of the last layer now after the pop
        pyramid[-1] += expended #Add it to the pyramid back
    print(len(pyramid))
    return pyramid[0]

### 1.2.4 Test your methods: `laplacian_reduce()`, `laplacian_expand()`

Run the following tests and make sure that the reconstruced image is identitcal to the original image.

In [ ]:
LaplacianExpandExample = namedtuple('LaplacianExpandExample', 'name path')
test_examples = [
    LaplacianExpandExample('Weizmann', 'data/pyramids/weizmann.png'),
    LaplacianExpandExample('Lenna', 'data/pyramids/lenna.png'),
    LaplacianExpandExample('Einstein', 'data/pyramids/einstein.png'),
]
examples_outputs = widgets.Accordion(children=[widgets.Output() for _ in range(len(test_examples))])

kernel = pyramid_kernel(a)
for i, example in enumerate(test_examples):
    # set example name
    examples_outputs.set_title(i, example.name)
    
    # plot inside example's area
    with examples_outputs.children[i]:
        image = imread(example.path)
        pyramid = laplacian_reduce(image, kernel, depth=depth)
        expanded = laplacian_expand(pyramid, kernel)
        imshow_tabs([expanded, image], tab_names=['Reconstructed', 'Original'])
        

display(examples_outputs)

# 2 Applications [50 Points]

## 2.1 Image Focusing [25 Points]
In this part you will experiment with image focusing using pyramids. The goal is to take two pictures that were shot with different focal planes and use them to generate a multi-focus image.
Open the article "Pyramid Methods in Image Processing" by Burt, Adelson et al. (1984) which is included in this exercise (under `papers/`), and read about image focusing (page 38, blue box). Please note that some errata were corrected for you (marked in red).

**Note:** The article addresses gray level images. When focusing RGB images, the correct way is to determine the maximum by the intensity image i.e. use `rgb2grey()` of the image (The final result should be in RGB).

### 2.1.1 Theoretical Question [10 points]
What is the main idea that underlies this method of image focusing? Why should we expect it to work? A brief and concise explanation should suffice.

No focus is "no high frequencies" = the frequencies are moderated.

we can combine two photos by taking the high (absolute) values of the laplacian pyramid, because the laplacian pyramid reserve well high frequencies and they are "continue" with us to the laplasian pyramid

### 2.1.2 Implement `image_focus()` [15 points]

Implement `image_focus()`  that receives two unfocused images and combines them into a single focused image. The result should match the focused parts of the given images. You're provided with tabs that can help you flicker between focused/unfocused images to test your result.

**Note:** The correct way to combine the coarsest level (the "gaussian" level) of the two pyramids is to *average* them, and not to handle them as the finer ("laplacian") levels.

In [ ]:
def image_focus(im1, im2, depth, a):
    """Generate a focused image from two unfocused images
    
    Argls:
        im1 (np.ndarray): unfocused image #1
        im2 (np.ndarray): unfocused image #2
        depth (int): depth of pyramids to use
        a (float): kernel parameter
    
    Returns:
        np.ndarray: a focused image
    """
    # YOUR CODE HERE
    
    
    kernel = pyramid_kernel(a) #Produce kernel with parameter a - []
    laplas1 = laplacian_reduce(im1,kernel,depth) #Laplacian on 1st img
    laplas2 = laplacian_reduce(im2,kernel,depth) #Laplacian on 2nd image
    new_laplas_combined = [] #The combined laplacian pyramid we will create
    for i in range(laplas1.__len__()):
        greyed_1 = rgb2grey(laplas1[i]) # grey the image out
        greyed_2 = rgb2grey(laplas2[i])
        # as explained in the blue box , the choice depends on the question if
        # ** on a greyed out image **
        # |LAᵢ (i,j)| > |LBᵢ (i,j)|
        # where
        # -  LAᵢ - is the i layer in the first laplasian
        # -  LBᵢ - is the i layer in the second laplasian pyramid
        # located in i,j pixel
        # the decision is made for each pixel 
        # we will use the choice function for that
        term  = np.absolute(greyed_1) > np.absolute(greyed_2)
        # now, in order to work on the original matrix, we should have the term (which is now greyed out)
        # to work on any color channel - i.e, become 3 dimantional
        # we will make 3 copies of choice in each dimation
        term = np.repeat(term[:, :, np.newaxis], 3, axis=2)
        new_laplas_combined.append(np.where(term,laplas1[i],laplas2[i]))
    # now we are just left to expend the laplas pyramid to construct the original image
    return laplacian_expand(pyramid=new_laplas_combined,kernel=kernel)

    # for pyr1,pyr2 in zip(lap1,lap2): #Run over depths of both pyramids
    #     grey1 = rgb2grey(pyr1) #Greyscale of first image as said in the hint above
    #     grey2 = rgb2grey(pyr2) #Greyscale of second image
    # #     choice = np.absolute(grey1) > np.absolute(grey2) #Matrix indicating where to take from first image
    # #     choice = np.repeat(choice[:, :, np.newaxis], 3, axis=2) #Repeating matrix to the three color channels
    #     combinedLap.append(np.where(choice,pyr1,pyr2)) #Choosing appropriate image
    
    # return laplacian_expand(combinedLap,kernel) #Expanding by the constructed pyramid
    
    
     
    

### 2.1.3 Test your method: `image_focus()`

Run the following tests and make sure that `image_focus()` works well.

In [ ]:
FocusExample = namedtuple('FocusExample', 'name path1 path2')
test_examples = [
    FocusExample('Leopard', 'data/focus/leopard1.bmp', 'data/focus/leopard2.bmp'),
    FocusExample('Pumpkin', 'data/focus/pumpkin1.png', 'data/focus/pumpkin2.png'),
    FocusExample('Tree', 'data/focus/tree1.png', 'data/focus/tree2.png'),
    FocusExample('Einstein', 'data/focus/einstein1.png', 'data/focus/einstein2.png'),
    FocusExample('Mountain', 'data/focus/mountain1.png', 'data/focus/mountain2.png'),
]
examples_outputs = widgets.Accordion(children=[widgets.Output() for _ in range(len(test_examples))])

kernel = pyramid_kernel(a)
for i, example in enumerate(test_examples):
    # set example name
    examples_outputs.set_title(i, example.name)
    
    # plot inside example's area
    with examples_outputs.children[i]:
        im1 = imread(example.path1)
        im2 = imread(example.path2)
        focused = image_focus(im1, im2, depth=depth, a=a)
        images = [np.vstack([img, (focused - img) / 2 + 0.5]) for img in [focused, im1, im2]]
        tab_names = ['Focused', 'Image 1', 'Image 2']
        imshow_tabs(images, tab_names, titles='`image` (top), `focused - image` (bottom)')

display(examples_outputs)

### 2.1.4 (Bonus) Implement `image_multi_focus()` [+10 points]

Generalize the algorithm for image focusing, and implement `image_multi_focus()` that takes *multiple* (more than 2) unfocused images, and combines them into a single focused image.

In [ ]:
def image_multi_focus(images, depth, a):
    """Generate a focused image from multiple unfocused images
    
    Args:
        images (List[np.ndarray]): list of unfocused images
        depth (int): depth of pyramids to use
        a (float): kernel parameter
    
    Returns:
        np.ndarray: a focused image
    """
    # YOUR CODE HERE

### 2.1.5 (Bonus) Test your method: `image_multi_focus()`

Run the following tests and make sure that `image_multi_focus()` works well.

In [ ]:
MultiFocusExample = namedtuple('MultiFocusExample', 'name paths')
test_examples = [
    MultiFocusExample('Einsten', ['data/multi_focus/einstein%d.png' % (i + 1) for i in range(4)]),
    MultiFocusExample('Tree', ['data/multi_focus/tree%d.png' % (i + 1) for i in range(4)]),
    MultiFocusExample('Mountain', ['data/multi_focus/mountain%d.png' % (i + 1) for i in range(4)]),
]
examples_outputs = widgets.Accordion(children=[widgets.Output() for _ in range(len(test_examples))])

try:
    kernel = pyramid_kernel(a)
    for i, example in enumerate(test_examples):
        # set example name
        examples_outputs.set_title(i, example.name)

        # plot inside example's area
        with examples_outputs.children[i]:
            imgs = [imread(path) for path in example.paths]
            focused = image_multi_focus(imgs, depth=depth, a=a)
            images = [np.vstack([img, focused - img + 0.5]) for img in [focused] + imgs]
            tab_names = ['Focused'] + ['Image %d' % (i + 1) for i in range(len(imgs))]
            imshow_tabs(images, tab_names, titles='`image` (top), `focused - image` (bottom)')

    display(examples_outputs)

except NotImplementedError:
    print("You should solve the bonus in order to test it.")

## 2.2 Image Mosaicing [25 points]
In this part you will get to know a surprising application of image pyramids. It turns out that we can stitch images together in a visually-appealing way if we make the transition separately at each level of the Laplacian pyramid.
First, read about image mosaicing in pages 39-40 (green boxes) on the article "Pyramid Methods in Image Processing" by Burt, Adelson et al. (1984).

### 2.2.1 Theoretical Question [10 points]
What is the main idea of using the Laplacian pyramid for performing image mosaicing? Why should this work better than using a simple smooth transition of the form:
\begin{equation*}
I_{out} = \left( 1-m \right) \cdot I_1 + m \cdot I_2
\end{equation*}

where $𝑚$ is a 2D mask with values between 0 and 1 (generated by smoothing a binary mask)?

The width through which the mask is smoothed is an external parameter which is constant. In this approach it mixes through high and low frequencies in the same manner, where we would prefer for finer transitions for higher frequencies. This is what the approach through the laplacian image gives us - each frequency is mixed with a matching width.

### 2.2.2 Implement `image_mosaic()` [15 points]
Implement the function `image_mosaic()` that gets two images `im1` and `im2`, and a 2D mask `mask` (with values in [0,1]). The function should stitch the images, according to the mask.

**Note:** The article uses a binary mask in order to choose the value at each point in the pyramid. However, in order to avoid sharp transitions and get appealing results, we need to smooth the mask a bit at each pyramid level. Smooth the `mask` with `5x5` uniform filter before downscaling it.

In [ ]:
def image_mosaic(im1, im2, mask, depth, a):
    """Stich two images together
    
    Args:
        im1 (np.ndarray): first image to stich (where `mask=0`)
        im2 (np.ndarray): second image to stich (where `mask=1`)
        mask (np.ndarray): mask to select what part to take from which image
        depth (int): how many pyramid levels to use in for smooth result
        a (float): kernel parameter
    
    Returns:
        np.ndarray: stiched images according to mask
    """
    # YOUR CODE HERE
    # this part is in the last green box in the second article
    # we sould fitst build the laplasian of both images and then create new one based on them
    # for the laplas we should have a kernel
    kernel = pyramid_kernel(a)
    laplas1 = laplacian_reduce(im1,kernel,depth)
    laplas2 = laplacian_reduce(im2,kernel,depth)
    uniform = pyramid_kernel(a=0) #Uniform 5X5 mask that sums up to 1 
    new = [] #The combined pyramid we will construct
    for i in range(depth + 1): #Run over all the depths plus one
    # 😑 failed on the tests where the depth was zero (one layer... :-( ))
    #     # instead of 'if where mask = 1 or mask =0, we can 'convolve them' like this'
    #     # where mask=1 laplas2 should be used
        new.append(mask*(laplas2[i]) + (1-mask)*(laplas1[i])) 
        mask = conv2d(mask,uniform) #Smoothen the mask
        mask = mask[::2,::2,:] #sub the mask for the next layer
    
    return laplacian_expand(new,kernel) #Reconstruct the new image after masking

### 2.2.3 Test your method: `image_mosaic()`

Run the following tests and make sure that `image_mosaic()` works well.

In [ ]:
MosaicExample = namedtuple('MultiFocusExample', 'name path1 path2 path_mask')
test_examples = [
    MosaicExample('Orange + Apple',
                  'data/mosaic/orange_apple/orange.jpg',
                  'data/mosaic/orange_apple/apple.jpg',
                  'data/mosaic/orange_apple/orangapple_m.bmp'),
    MosaicExample('Monalisa + Obama',
                  'data/mosaic/monalisa_obama/monalisa.jpg',
                  'data/mosaic/monalisa_obama/obama.jpg',
                  'data/mosaic/monalisa_obama/monobama_m.jpg'),
    MosaicExample('Michal + Tal',
                  'data/mosaic/michal_tal/michal.bmp',
                  'data/mosaic/michal_tal/tal.bmp',
                  'data/mosaic/michal_tal/michtal_m.bmp'),
]
examples_outputs = widgets.Accordion(children=[widgets.Output() for _ in range(len(test_examples))])

kernel = pyramid_kernel(a)
for i, example in enumerate(test_examples):
    # set example name
    examples_outputs.set_title(i, example.name)
    
    # plot inside example's area
    with examples_outputs.children[i]:
        im1 = imread(example.path1)
        im2 = imread(example.path2)
        mask = imread(example.path_mask)
        mosaic_naive = image_mosaic(im1, im2, mask, depth=0, a=a)
        mosaic = image_mosaic(im1, im2, mask, depth=depth, a=a)
        images = [mosaic, mosaic_naive, im1, im2, mask]
        tab_names = ['Mosaic', 'Naive Mosaic', 'Image 1', 'Image 2', 'Mask']
        imshow_tabs(images, tab_names)
        
display(examples_outputs)